# **Evaluation**

First, we will install the necessary libraries and adding groq api kre.

In [4]:
## ! pip install langchain langchain_groq langchain_community

In [55]:
import os

os.environ["GROQ_API_KEY"] = "YOUR-API-KEY"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

- Loading the document

In [9]:
from langchain.document_loaders import CSVLoader

file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path = file)

document = loader.load()

- Setup the LLM

In [57]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature = 0.0,
    model = "llama3-70b-8192"
)

- In next step we will split the text

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

docs = text_splitter.split_documents(document)

In [15]:
## ! pip install sentence-transformers chromadb

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(docs, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [58]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vectordb.as_retriever(search_kwargs = {"k": 6}),
    verbose = True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [59]:
from IPython.display import display, Markdown

display(Markdown(document[0].page_content))

: 0
name: Women's Campside Oxfords
description: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. 

Size & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. 

Specs: Approx. weight: 1 lb.1 oz. per pair. 

Construction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. 

Questions? Please contact us for any inquiries.

In [60]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

## `QAGenerationChain`

`QAGenerateChain` is used to generate QA pairs from a given text. This is particularly useful for creating datasets for training or evaluating QA models, automating the generation of questions from textual data, or developing QA systems.

In [61]:
from langchain.evaluation.qa import QAGenerateChain

example_gen_chain = QAGenerateChain.from_llm(
    ChatGroq(
        model = "llama3-70b-8192",
        temperature = 0.0
    )
)

The code creates a list of dictionaries where each dictionary contains a key `doc` with a value corresponding to each of the first five elements in the document list. This is done using a list comprehension.
The `apply_and_parse` method of `example_gen_chain` is called with this list of dictionaries. This method is responsible for generating QA pairs from the given documents and parsing the results.


In [62]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in document[:5]]
)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [63]:
new_examples

[{'qa_pairs': {'query': "What is the recommended sizing approach for half sizes that are not offered in the Women's Campside Oxfords?",
   'answer': 'Order up to the next whole size.'}},
 {'qa_pairs': {'query': 'What percentage of recycled materials are used to make the 24 oz. polyester fabric of the Recycled Waterhog dog mat?',
   'answer': '94%'}},
 {'qa_pairs': {'query': "What percentage of the sun's harmful rays does the fabric of the Infant and Toddler Girls' Coastal Chill Swimsuit block, according to its UPF 50+ rating?",
   'answer': '98%'}},
 {'qa_pairs': {'query': 'What percentage of Lycra spandex is used in the body of the Refresh Swimwear V-Neck Tankini?',
   'answer': '18%'}},
 {'qa_pairs': {'query': 'What is the primary benefit of the TEK O2 technology used in the EcoFlex 3L Storm Pants, according to the product description?',
   'answer': 'The primary benefit of the TEK O2 technology is that it provides enhanced breathability, making the pants more comfortable and dry in 

In [64]:
document[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [65]:
## Add the new generated questions to our list
for eg in new_examples:
    examples.append(eg['qa_pairs'])

examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What is the recommended sizing approach for half sizes that are not offered in the Women's Campside Oxfords?",
  'answer': 'Order up to the next whole size.'},
 {'query': 'What percentage of recycled materials are used to make the 24 oz. polyester fabric of the Recycled Waterhog dog mat?',
  'answer': '94%'},
 {'query': "What percentage of the sun's harmful rays does the fabric of the Infant and Toddler Girls' Coastal Chill Swimsuit block, according to its UPF 50+ rating?",
  'answer': '98%'},
 {'query': 'What percentage of Lycra spandex is used in the body of the Refresh Swimwear V-Neck Tankini?',
  'answer': '18%'},
 {'query': 'What is the primary benefit of the TEK O2 technology used in the EcoFlex 3L Storm Pants, according to the product descriptio

In [66]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'According to the provided context, yes, the Cozy Comfort Pullover Set has side pockets. It is mentioned in the "Additional Features" section: "Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg."'

# **Manual Evaluation**

The `langchain.debug` module in LangChain is designed to facilitate debugging and logging during the development and execution of language model applications. It helps developers track the flow of data, inspect intermediate results, and diagnose issues more effectively. Here's a detailed overview of its usage and features:

In [67]:
import langchain
langchain.debug = True

In [68]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": "Chest pockets with snap closures. Pencil pocket on chest. Underarm gussets enhance mobility. Berber fleece-lined pockets keep hands warm. One pocket has a hidden security pocket tucked inside. Imported.<<<<>>>>>: 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit si

'According to the provided context, yes, the Cozy Comfort Pullover Set has side pockets. It is mentioned in the "Additional Features" section: "Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg."'

In [69]:
# Turn off the debug mode
langchain.debug = False

## **LLM assisted evaluation**

The `QAEvalChain` in `langchain.evaluation.qa` is an essential tool for evaluating the performance of QA models. By comparing the model's answers to reference answers and computing various evaluation metrics, it provides valuable insights into the accuracy and quality of the QA system. This helps in validating, benchmarking, and improving QA models, ensuring they deliver accurate and relevant answers.

In [70]:
examples[0]['query']

'Do the Cozy Comfort Pullover Set        have side pockets?'

In [73]:
from langchain.evaluation.qa import QAEvalChain

## generates predictions for each example in examples.
predictions = qa.apply(examples)

llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0.0
)

eval_chain = QAEvalChain.from_llm(llm)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [74]:
## Evaluate the quality of predicited answers for examples
graded_outputs = eval_chain.evaluate(examples, predictions)

Finally, let's go through the whole code from begginging and see how the results are:

In [75]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: According to the provided context, yes, the Cozy Comfort Pullover Set has side pockets. It is mentioned in the "Additional Features" section: "Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg."

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.

Example 2:
Question: What is the recommended sizing approach for half sizes that are not offered in the Women's Campside Oxfords?
Real Answer: Order up to the next whole size.
Predicted Answer: According to the product description, for half sizes not offered in the Women's Campside Oxfords, it is recommended to order up to the next whole size.

Example 3:
Question: What percentage of recycled materials are